In [34]:
import pandas as pd
attendance = pd.read_csv("https://data.cityofnewyork.us/api/views/7z8d-msnt/rows.csv?accessType=DOWNLOAD")
attendance.set_index('District')[:-1].to_csv("../data/school_attendance.csv")

In [35]:
attendance.head()

,District,YTD % Attendance (Avg),YTD Enrollment(Avg)
0,DISTRICT 01,91.18%,12367
1,DISTRICT 02,89.01%,60823
2,DISTRICT 03,89.28%,21962
3,DISTRICT 04,91.13%,14252
4,DISTRICT 05,89.08%,13170


In [36]:
len(attendance)

35

In [3]:
%load_ext py_d3

In [53]:
%%d3 4.4.4

<svg style="width:800px; height:400px"></svg>

<script>
    d3.csv("../data/school_attendance.csv", function(data) {
        // Use d3.pie() to create and configure the data that we need in a format that we can enter into.
        let arc_data = d3.pie().value(d => d['YTD Enrollment(Avg)']).padAngle(d => 0.0115)(data);
        
        // Create the arc factory function that will render each data segment.
        let arc = d3.arc().innerRadius(75).outerRadius(160);

        // Run through each element of arc_data, creating and appening the arc for each one.
        d3.select("svg")
            .append("g")
            .attr("id", "transform")
            .attr("transform", "translate(400, 200)")
            .selectAll('path')
            .data(arc_data)
            .enter()
            .append('path')
            .attr('d', arc)
            .attr('fill', 'steelblue');
        
        // Use arc and arc_data to calculate centroids, and from there to calculate.
        arc_data.forEach(function(d, i) {
            [x, y] = arc.centroid(d);
            let label = d.data['District']
            let rotation = d['startAngle'] / Math.PI / 2 * 360 - 90;
            d3.select("#transform").append("text")
                .attr("x", x).attr("y", y)
                .attr("text-anchor", "middle").attr("alignment-baseline", "middle")
                .attr("transform", "rotate(" + rotation + ")")
                .text(label);
        })
    })
</script>

Note how this doesn't work. To demonstrate, I've generated the rotation of each label so that it is 5 times its indicial location. As you can see, the rotations do not go around the pie in a well-sorted order.

Try replacing `d.index` with `i`, and things become even more garbled. This is because `i` is the pre-sorted order of the data points, e.g. the order in which they appeared *before* they were sorted in ascending order by `d3.pie()` (resulting in the neat gradient depicted here).

In [85]:
%%d3 4.4.4

<svg style="width:800px; height:400px"></svg>

<script>
    d3.csv("../data/school_attendance.csv", function(data) {
        // Use d3.pie() to create and configure the data that we need in a format that we can enter into.
        let arc_data = d3.pie().value(d => d['YTD Enrollment(Avg)']).padAngle(d => 0.0115)(data);
        
        // Create the arc factory function that will render each data segment.
        let arc = d3.arc().innerRadius(75).outerRadius(160);

        // Run through each element of arc_data, creating and appening the arc for each one.
        d3.select("svg").selectAll('path')
            .data(arc_data)
            .enter()
            .append('path')
            .attr('d', arc)
            .attr('fill', 'steelblue').attr("transform", "translate(400, 200)");
        
        // Use arc and arc_data to calculate centroids, and from there to calculate.
        arc_data.forEach(function(d, i) {
            [x, y] = arc.centroid(d);
            let label = d.data['District']
            let rotation = d['startAngle'] / 2 * Math.PI * 360 - 90;
            d3.select("svg").append("text")
                .attr("x", x).attr("y", y)
                .attr("text-anchor", "middle").attr("alignment-baseline", "middle")
                .attr("transform", "translate(400, 200) rotate(" + rotation + ")").text(label);
            console.log(d.data['District'], rotation)
        })
    })
</script>

Why does this happen? Read more here: http://stackoverflow.com/questions/28884446/d3-rotating-text-around-a-pie-chart